In [60]:
from selenium import webdriver
import yfinance as yf
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

In [61]:
ticker = 'BBAS3.SA'
acao = yf.Ticker(ticker)
info = acao.info


In [83]:
historico = acao.history(period='10y')
historico

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-11-18 00:00:00-02:00,6.794376,7.006533,6.708439,6.982363,10429800,0.0,0.0
2014-11-19 00:00:00-02:00,6.985050,7.452331,6.985050,7.395936,20641600,0.0,0.0
2014-11-21 00:00:00-02:00,7.522154,8.037775,7.522154,8.010920,25128000,0.0,0.0
2014-11-24 00:00:00-02:00,8.104912,8.351980,7.691341,7.731624,17509000,0.0,0.0
2014-11-25 00:00:00-02:00,7.788019,8.010917,7.785333,7.844415,16434200,0.0,0.0
...,...,...,...,...,...,...,...
2024-11-11 00:00:00-03:00,26.100000,26.100000,25.950001,26.010000,10515100,0.0,0.0
2024-11-12 00:00:00-03:00,26.010000,26.090000,25.799999,25.950001,15391500,0.0,0.0
2024-11-13 00:00:00-03:00,25.990000,26.070000,25.879999,25.950001,27198100,0.0,0.0


In [88]:
dividendos = acao.dividends
historico_com_dividendos = historico[historico.index.isin(dividendos.index)]
historico_com_dividendos['DY'] =  (historico_com_dividendos['Dividends'] / historico_com_dividendos['Open'])*100
historico_com_dividendos.head(60)

C:\Users\vitor\AppData\Local\Temp\ipykernel_22676\879929311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historico_com_dividendos['DY'] =  (historico_com_dividendos['Dividends'] / historico_com_dividendos['Open'])*100


,Open,High,Low,Close,Volume,Dividends,Stock Splits,DY
Date,,,,,,,,
2014-12-12 00:00:00-02:00,6.667700,6.716707,6.324649,6.324649,14275200,0.169975,0.0,2.549230
2015-02-20 00:00:00-02:00,6.690034,6.690034,6.490536,6.550658,5348200,0.045600,0.0,0.681611
2015-03-24 00:00:00-03:00,6.608172,6.649820,6.466569,6.533206,8084600,0.188496,0.0,2.852468
2015-05-22 00:00:00-03:00,6.759755,6.782381,6.604196,6.604196,16918400,0.225569,0.0,3.336941
2015-06-12 00:00:00-03:00,6.499193,6.582259,6.444770,6.444770,13196600,0.144947,0.0,2.230231
2015-08-24 00:00:00-03:00,5.022117,5.336721,4.941302,5.175090,24489600,0.070520,0.0,1.404189
2015-09-14 00:00:00-03:00,4.779282,5.031595,4.758744,5.011058,17450400,0.133039,0.0,2.783661
2015-11-24 00:00:00-02:00,5.418832,5.552083,5.332961,5.507666,11256800,0.085403,0.0,1.576041
2015-12-14 00:00:00-02:00,5.221698,5.227713,4.999114,5.005129,16431200,0.137246,0.0,2.628379


In [56]:

for ano in range(dividendos.index[0].year, dividendos.index[-1].year + 1):
    # Filtra os dividendos do ano específico
    dividendos_ano = dividendos[dividendos.index.year == ano].sum()
    
    # Obtém o preço de fechamento no início do ano
    preco_inicio_ano = historico.loc[f'{ano}-01-01':'{ano}-01-31']['Close'].iloc[0]

    # Calcula o dividend yield anual
    dividend_yield_anual = (dividendos_ano / preco_inicio_ano) * 100

# Exibindo o dividend yield anual
for ano, yield_ in dividend_yield_anual.items():
    print(f"Dividend Yield de {ano}: {yield_:.2f}%")

TypeError: cannot do slice indexing on DatetimeIndex with these indexers [{ano}-01-31] of type str

In [22]:
url = 'https://api.mziq.com/mzfilemanager/v2/d/5760dff3-15e1-4962-9e81-322a0b3d0bbd/5b90cdd9-0fb8-aaab-afbb-10ee2a4933ed?origin=1'
response = requests.get(url)
print(response)

<Response [200]>
